In [1]:
import os
import sys

current_dir = os.getcwd()

parent_dir = os.path.abspath(os.path.join(current_dir, ".."))

if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [3]:
from treemort.utils.config import setup

config_file_path = "../configs/flair_unet_bs8_cs256.txt"
conf = setup(config_file_path)

# Modified Config Variables for Local Execution; comment on HPC
conf.data_folder = "/Users/anisr/Documents/AerialImages"
conf.output_dir = os.path.join("..", conf.output_dir)

print(conf)

Namespace(data_folder='/Users/anisr/Documents/AerialImages', hdf5_file='AerialImageModel_ITD.h5', model='flair_unet', backbone=None, epochs=100, train_batch_size=8, val_batch_size=8, test_batch_size=8, train_crop_size=256, val_crop_size=256, test_crop_size=256, val_size=0.2, test_size=0.1, input_channels=4, output_channels=1, output_dir='../output/flair_unet', model_weights='best', learning_rate=0.0001, threshold=0.5, activation='sigmoid', loss='hybrid', resume=True)


In [2]:
import cv2
import torch
import geojson
import rasterio

import numpy as np

from tqdm import tqdm
from shapely.geometry import Polygon
from concurrent.futures import ThreadPoolExecutor

In [3]:
id2label = {0: "alive", 1: "dead"}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

from treemort.modeling.builder import build_model

print("[INFO] Loading or resuming model...")
model, optimizer, criterion, metrics = build_model(conf, id2label, device)
print(f"[INFO] Model, optimizer, criterion, and metrics are set up.")

model = model.to(device)

checkpoint_path = "/Users/anisr/Documents/TreeSeg/output/hcfnet/best.weights.pth"

model.load_state_dict(torch.load(checkpoint_path, map_location=device))
print(f"[INFO] Loaded weights from {checkpoint_path}.")

[INFO] Using device: cpu


/Users/anisr/Documents/TreeSeg/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[INFO] Loading or resuming model...


NameError: name 'conf' is not defined

In [36]:
import numpy as np
import torch
from tqdm import tqdm

def load_and_preprocess_image(tiff_file):
    with rasterio.open(tiff_file) as src:
        image = src.read()

        image = image.astype(np.float32) / 255.0

        transform = src.transform

    image = np.transpose(image, (1, 2, 0))  # From (C, H, W) to (H, W, C)

    return image, transform


def sliding_window_inference(model, image, window_size, stride, device, batch_size=8):
    model.eval()  # Ensure model is in evaluation mode

    # Pad the image to handle edge cases
    padded_image = pad_image(image, window_size)

    h, w = padded_image.shape[:2]
    prediction_map = np.zeros((h, w), dtype=np.float32)
    count_map = np.zeros((h, w), dtype=np.float32)

    # List to hold patches for batching
    patches = []

    # Initialize progress bar
    total_patches = ((h - window_size) // stride + 1) * ((w - window_size) // stride + 1)
    with tqdm(total=total_patches, desc="Processing patches") as pbar:
        
        for y in range(0, h - window_size + 1, stride):
            for x in range(0, w - window_size + 1, stride):
                image_patch = padded_image[y:y + window_size, x:x + window_size]

                # Add the patch to the list
                patches.append((y, x, image_patch))

                # Process the batch if it reaches the specified size
                if len(patches) == batch_size:
                    process_batch(model, patches, prediction_map, count_map, device)
                    patches = []  # Clear the list for the next batch

                pbar.update(1)

        # Process any remaining patches in the list
        if patches:
            process_batch(model, patches, prediction_map, count_map, device)

    # Normalize prediction map by count map
    with np.errstate(divide='ignore', invalid='ignore'):
        prediction_map /= count_map
        prediction_map[count_map == 0] = 0  # Handle divisions by zero

    # Crop prediction_map to the original image size
    if isinstance(image, tuple):
        image = image[0]

    prediction_map = prediction_map[:image.shape[0], :image.shape[1]]

    return prediction_map

def process_batch(model, patches, prediction_map, count_map, device):
    # Convert patches to tensors
    batch_patches = [torch.from_numpy(patch[2]).permute(2, 0, 1).unsqueeze(0).float().to(device) for patch in patches]
    batch_patches_tensor = torch.cat(batch_patches, dim=0)  # Create batch tensor

    # Perform inference
    with torch.no_grad():
        outputs = model(batch_patches_tensor)

    # Process each patch in the batch
    for (y, x, _), output in zip(patches, outputs):
        prediction = output.squeeze(0).squeeze(0).cpu().numpy()
        prediction_map[y:y + batch_patches_tensor.shape[2], x:x + batch_patches_tensor.shape[3]] += prediction
        count_map[y:y + batch_patches_tensor.shape[2], x:x + batch_patches_tensor.shape[3]] += 1




    
'''
import numpy as np
import torch
import torch.multiprocessing as mp

# Define process_patch at the top level
def process_patch(model, image_patch, device):
    image_patch_tensor = torch.from_numpy(image_patch).permute(2, 0, 1).unsqueeze(0).float().to(device)
    with torch.no_grad():
        output = model(image_patch_tensor)
    prediction = output.squeeze(0).squeeze(0).cpu().numpy()
    return prediction

def sliding_window_inference(model, image, window_size, stride, device):
    model.eval()
    padded_image = pad_image(image, window_size)

    h, w = padded_image.shape[:2]
    prediction_map = np.zeros((h, w), dtype=np.float32)
    count_map = np.zeros((h, w), dtype=np.float32)

    patches = [(padded_image[y : y + window_size, x : x + window_size], device)
               for y in range(0, h - window_size + 1, stride)
               for x in range(0, w - window_size + 1, stride)]

    # Use multiprocessing Pool
    with mp.Pool(mp.cpu_count()) as pool:
        results = pool.starmap(process_patch, [(model, p, device) for p in patches])

    # Aggregate results back to the prediction map
    idx = 0
    for y in range(0, h - window_size + 1, stride):
        for x in range(0, w - window_size + 1, stride):
            prediction = results[idx]
            prediction_map[y : y + window_size, x : x + window_size] += prediction
            count_map[y : y + window_size, x : x + window_size] += 1
            idx += 1

    with np.errstate(divide='ignore', invalid='ignore'):
        prediction_map /= count_map
        prediction_map[count_map == 0] = 0

    if isinstance(image, tuple):
        image = image[0]

    prediction_map = prediction_map[:image.shape[0], :image.shape[1]]

    return prediction_map

'''

def pad_image(image, window_size):
    if isinstance(image, tuple):
        image = image[0]

    h, w = image.shape[:2]
    pad_h = max(0, window_size - h)
    pad_w = max(0, window_size - w)
    
    return np.pad(image, ((0, pad_h), (0, pad_w), (0, 0)), mode="constant")


def threshold_prediction_map(prediction_map, threshold=0.5):
    binary_mask = (prediction_map >= threshold).astype(np.uint8)
    return binary_mask


def extract_contours(binary_mask):
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours


def apply_transform(contour, transform):
    transformed_contour = np.array([transform * (x, y) for x, y in contour])
    return transformed_contour


def contours_to_geojson(contours, transform, name="M4124C_2017_1", crs="urn:ogc:def:crs:EPSG::3067"):
    features = []
    
    for contour in contours:
        if len(contour) >= 3:  # Ensure valid contour (at least 3 points)
            contour = contour.reshape(-1, 2)

            contour = apply_transform(contour, transform)

            if not np.array_equal(contour[0], contour[-1]):
                contour = np.vstack([contour, contour[0]])

            polygon = Polygon(contour)
            feature = {
                "type": "Feature",
                "properties": {
                    "id": None,  # Add any properties you need here
                    "max_value": None
                },
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [contour.tolist()]
                }
            }
            features.append(feature)
        else:
            print(f"Skipped contour with {len(contour)} points")

    geojson = {
        "type": "FeatureCollection",
        "name": name,
        "crs": {
            "type": "name",
            "properties": {
                "name": crs
            }
        },
        "features": features
    }
    
    return geojson


def save_geojson(geojson_data, output_path):
    with open(output_path, 'w') as f:
        geojson.dump(geojson_data, f)

In [38]:
import os

def process_image(image_path, window_size=256, stride=128, threshold=0.5):
    print(f"[INFO] Starting process for image: {image_path}")
    
    image, transform = load_and_preprocess_image(image_path)
    print(f"[INFO] Image loaded and preprocessed. Shape: {image.shape}, Transform: {transform}")
    
    prediction_map = sliding_window_inference(model, image, window_size, stride, device)
    print(f"[INFO] Prediction map generated with shape: {prediction_map.shape}")
    
    binary_mask = threshold_prediction_map(prediction_map, threshold)
    print(f"[INFO] Binary mask created with threshold: {threshold}. Mask shape: {binary_mask.shape}")
    
    contours = extract_contours(binary_mask)
    print(f"[INFO] {len(contours)} contours extracted from binary mask")
    
    geojson_data = contours_to_geojson(contours, transform)
    print(f"[INFO] Contours converted to GeoJSON format")
    
    output_geojson_path = os.path.join(os.path.dirname(image_path), os.path.splitext(os.path.basename(image_path))[0] + "_pred.geojson")
    save_geojson(geojson_data, output_geojson_path)
    print(f"[INFO] GeoJSON saved to {output_geojson_path}")


image_path = "/Users/anisr/Documents/TreeSeg/demo/files/M4124C_2017_1.tiff"
image_path = "/Users/anisr/Documents/TreeSeg/demo/files/T4323E_1.tif"

process_image(image_path)

[INFO] Starting process for image: /Users/anisr/Documents/TreeSeg/demo/files/T4323E_1.tif
[INFO] Image loaded and preprocessed. Shape: (5882, 4756, 4), Transform: | 0.25, 0.00, 440000.00|
| 0.00,-0.25, 7367234.50|
| 0.00, 0.00, 1.00|


Processing patches: 100%|██████████| 1584/1584 [02:25<00:00, 10.90it/s]

[INFO] Prediction map generated with shape: (5882, 4756)
[INFO] Binary mask created with threshold: 0.5. Mask shape: (5882, 4756)
[INFO] 313 contours extracted from binary mask
Skipped contour with 1 points
Skipped contour with 2 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 2 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 2 points
Skipped contour with 2 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 2 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 2 points
Skipped contour with 2 points
Skipped contour with 2 points
Skipped contour with 2 points
Skipped contour with 1 points
Skipped contour with 1 points
Skipped contour with 2 points
Skipped contour with 1 points
Skipped contour with 2 points
[INFO] Contou

In [43]:
import json
import geopandas as gpd
from shapely.geometry import shape
from shapely.ops import unary_union

def calculate_iou(true_geojson, pred_geojson):
    # Load the GeoJSON files
    true_gdf = gpd.GeoDataFrame.from_features(true_geojson["features"])
    pred_gdf = gpd.GeoDataFrame.from_features(pred_geojson["features"])

    intersection_area = 0.0
    union_area = 0.0

    for true_polygon in true_gdf.geometry:
        for pred_polygon in pred_gdf.geometry:
            if true_polygon.intersects(pred_polygon):
                intersection = true_polygon.intersection(pred_polygon)
                union = true_polygon.union(pred_polygon)
                
                intersection_area += intersection.area
                union_area += union.area

    if union_area == 0:
        return 0.0
    iou = intersection_area / union_area
    return iou

if __name__ == "__main__":
    with open("/Users/anisr/Documents/TreeSeg/demo/files/M4124C_2017_1.geojson") as f:
        true_geojson = json.load(f)
    
    with open("/Users/anisr/Documents/TreeSeg/demo/files/M4124C_2017_1_pred.geojson") as f:
        pred_geojson = json.load(f)
    
    iou_score = calculate_iou(true_geojson, pred_geojson)
    print(f"IoU Score: {iou_score:.4f}")

IoU Score: 0.0000


In [ ]:
import numpy as np
import cv2
from tqdm import tqdm

def calculate_iou_from_topo(true_topo, pred_topo, threshold=128):
    # Convert topological maps to binary masks
    true_binary = (true_topo >= threshold).astype(np.uint8)
    pred_binary = (pred_topo >= threshold).astype(np.uint8)

    # Calculate intersection and union
    intersection = np.logical_and(true_binary, pred_binary).sum()
    union = np.logical_or(true_binary, pred_binary).sum()

    # Compute IoU
    iou = intersection / union if union != 0 else 0.0
    return iou

# Example usage:
if __name__ == "__main__":
    # Generate segmentation maps (replace contours with actual data)
    true_topo = segmap_to_topo(true_image_np, true_contours)
    pred_topo = segmap_to_topo(pred_image_np, pred_contours)

    # Calculate IoU score
    iou_score = calculate_iou_from_topo(true_topo, pred_topo)
    print(f"IoU Score: {iou_score:.4f}")
